## Q1.

Use github to integrate our math library with Travis CI and Coveralls.

In [5]:
from IPython.core.display import HTML
HTML('https://github.com/akhilketkar/travisTest')

## Q2.

Take the implementation of binary search from a previous lecture/lab. Write unit tests for the algorithm (remember we have doctests in there), stripping the doctests down to those that illustrate the interface for a user. How do these doctests deal with the concerns we had?

In [37]:
%%file binsearch.py
def binary_search(da_array: list, needle, left:int=0, right:int=-1) -> int:
    """
    An algorithm that operates in O(lg(n)) to search for an element
    in an array sorted in ascending order.
    
    Parameters
    ----------
    da_array : list
        a list of "comparable"items sorted in non-descending order
    needle: an item to find in the array; it may or may not
        be in the array
    left: int, optional
        the left index in the array to search from. Default 0
    right: int, optional
        the right index in the array to search to. Default is -1
        in which case we will use the end of the array `len(da_array) - 1`
        
    Returns
    -------
    index: int
        an integer representing the index of `needle` if found, and -1
        otherwise
        
    Notes
    -----
    PRE: `da_array` is sorted in non-decreasing order (thus items in
        `da_array` must be comparable: implement < and ==)
    POST: 
        - `da_array` is not changed by this function (immutable)
        - returns `index`=-1 if `needle` is not in `da_array`
        - returns an int `index ` in [0:len(da_array)] if
          `index` is in `da_array`
    INVARIANTS:
        - If `needle` in `da_array`, needle in `da_array[rangemin:rangemax]`
          is a loop invariant in the while loop below.
          
    Examples
    --------
    >>> input = list(range(10))
    >>> binary_search(input, 5)
    5
    >>> binary_search(input, 4.5)
    -1
    >>> binary_search(input, 10)
    -1
    >>> binary_search([5], 5)
    0
    >>> binary_search([5], 4)
    -1
    >>> import numpy as np
    >>> binary_search([1,2,np.inf], 2)
    1
    >>> binary_search([1,2,np.inf], np.inf)
    2
    >>> binary_search(input, 5, 1,3)
    -1
    >>> binary_search(input, 2, 1,3)
    2
    >>> binary_search(input, 2, 3, 1)
    -1
    >>> binary_search(input, 2, 2, 2)
    2
    >>> binary_search(input, 5, 2, 2)
    -1
    """
    
    
    if left==0:
        rangemin = 0
    else:
        rangemin = left
    if right==-1:
        rangemax=len(da_array) - 1
    else:
        rangemax=right
    while True:
        "needle in da_array => needle in da_array[rangemin:rangemax]"   
        if rangemin > rangemax:
            index = -1
            return index
        #If rangemin and rangemax are both very high we do not want overflow,
        #so get the midpoint like this:
        midpoint = rangemin + (rangemax - rangemin)//2
        if da_array[midpoint] > needle:#lower part
            rangemax = midpoint - 1
        elif da_array[midpoint] < needle:
            rangemin = midpoint + 1
        else:
            index = midpoint
            return index

############ TESTING ################
import unittest
class TestBinSearch(unittest.TestCase):
    def test_checkPresent(self):
        inp = range(10)
        self.assertEqual(binary_search(inp,4),4)
    
    def test_checkAbsent(self):
        inp = range(10)
        self.assertEqual(binary_search(inp,-10),-1)
    
    def test_checkLarge1(self):
        'left edge'
        inp = range(10000)
        self.assertEqual(binary_search(inp,0),inp.index(0))
    
    def test_checkLarge2(self):
        'right edge'
        inp = range(10000)
        self.assertEqual(binary_search(inp,9999),inp.index(9999))
        
    def test_badInput1(self):
        """ The array must consist of things that can be compared to the needle """
        inp = ['a',1,2,3,4,'g']
        with self.assertRaises(TypeError):
            binary_search(inp,'g')
            
    def test_badInput2(self):
        """ The first argument must be a list or something that can be interpreted as a list """
        with self.assertRaises(TypeError):
            binary_search(5,5)
    
    def test_strings(self):
        inp = 'abcdefghi'
        self.assertEqual(binary_search(inp,'c'),2)
    
    def test_empty(self):
        self.assertEqual(binary_search([],9999),-1)

Overwriting binsearch.py


In [38]:
!python -m unittest -v binsearch.py

test_badInput1 (binsearch.TestBinSearch)
The array must consist of things that can be compared to the needle ... ok
test_badInput2 (binsearch.TestBinSearch)
The first argument must be a list or something that can be interpreted as a list ... ok
test_checkAbsent (binsearch.TestBinSearch) ... ok
test_checkLarge1 (binsearch.TestBinSearch)
left edge ... ok
test_checkLarge2 (binsearch.TestBinSearch)
right edge ... ok
test_checkPresent (binsearch.TestBinSearch) ... ok
test_empty (binsearch.TestBinSearch) ... ok
test_strings (binsearch.TestBinSearch) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.001s

OK


In addition, we should be **systematic** about testing our code. You should at-least jave some tests like this:

1. We should test with wierd data, ie a wierd array: does it have NANs, is it numeric? Does it have 0 elelemts? 1 element? 2?...ie test the boundaries

2. Then think of how the needle relates to the above. Try needles less than or greter than the range in the sorted array, besides needles inbetween (in both cases the needle not being in the array). Try needles at the extremes of the array.

3. test the integration of 1 and 2 to make sure something has not gone wrong there.

Note: you can always compare your answers with linear search implemented in python.

For reference, here are some of our concerns from that lab:

#### What's happened to our issue from before?

- What if the value is not there in the array? What if it is there multiple times? what are we returning (why the -1). Are we consistent in our returning?

We return -1 if the element is not in the array. If it is there multiple times, we will return one of them: it is not defined which. We are consistent by always returning an int, choosing one which cannot be an index.

- What if rangemax is so high as to create overflow: 

We fixed it by using the difference and have documented it in the algorithm


- what types are we supporting? . 

It seems that as long as we have a notion of equals `==`, and a notion of `<` to support sorting we are set. We should document this.

- what happens if we have a NaN in our array? Infty?

If our preconditions are violated by the user, we can do anything. Doing it nicely might be costly. so we wont.


- what if da_array was not an array?

The user violated the pre-conditions. Anything could happen. We could check for a list
but yhen that would hurt a special class which implemented the python sequence protocol

- What happens if array is not sorted: 

The user violated the pre-conditions. We could return an error, violate post conditions. If we sort it we'd violate the o(lg(n)) notion. (fixing it seems dubious). Can we check if its sorted? This is naively O(n) and breaks our performance specifications. We can create a guard to terminate the array at more than n iterations for the infinite case or let the user have enough rope to hang themselves



## Q3.

Implement a mean and a median methods for the values (not the times) in the timeseries class (with you made last week. Feel free to use numpy functions to do this. Make sure you have tests for these methods, and that you handle the boundaries.

We have asked you to define these methods within these classes, but we also want you to think if these would be best calculated as functions which take a time-series as an argument instead. (to think about, not to hand in)

### This is in the [cs207projects](https://github.com/CS207Project/cs207project) repo.